# Pystac Tutorial
## Create and manipulate Spacenet Rio STAC

This tutorial shows how to create and manipulate STACs using pystac. It contains two parts:

1. Create Spacenet Rio STAC
    - Create (in memory) a pystac catalog of [Spacenet 1 imagery from the Rio De Janeiro AOI](https://spacenet.ai/rio-de-janeiro/) using data hosted in a public s3 bucket
    - Set uri's based on a root directory
    - Save the STAC locally
    
    
2. Create a new STAC with COGs and labels
    - Determine which images included in the STAC overlap with building labels
    - For has labeled buildings, create and save off both COGs and geojsons of building labels
    - Create an updated STAC that points to the new files and ony includes labeled scenes
    - Set uri's based on a root directory
    - Save the STAC locally

In [1]:
import sys
sys.path.append('..')

In [2]:
from datetime import datetime
from os.path import basename, join
from subprocess import call

import boto3
import geopandas as gpd
import rasterio
from pystac import (Catalog, Collection, Extent, Item, SpatialExtent,
                    TemporalExtent)
from shapely.geometry import Polygon, box

### Create Spacenet Rio STAC

Initialize a STAC for the Spacenet 1 dataset

In [3]:
spacenet = Catalog(id='spacenet', description='Spacenet 1 STAC')

Create a spatial extent object using the AOI geojson on S3

In [4]:
aoi = 's3://spacenet-dataset/AOIs/AOI_1_Rio/metadata/AOI_1_Rio_metadata_public_AOI_outline.geojson'
sp_extent = SpatialExtent(list(gpd.read_file(aoi)['geometry'][0].bounds))

The capture date for Spacenet 1 Rio imagery is unspecified in the dataset page so we will use October 22, 2015 as an example date. Create a python datetime object using that date

In [5]:
capture_date = datetime.strptime('2015-10-22', '%Y-%m-%d') 
tmp_extent = TemporalExtent([(capture_date, None)])

Create an Extent object that will define both the spatial and temporal extents of the Rio collection

In [6]:
extent = Extent(sp_extent, tmp_extent)

Create a collection that will encompass all of the Rio De Janeiro data and add to the spacenet catalog

In [7]:
rio = Collection(id='rio', description = 'Rio Spacenet 3 dataset', extent = extent)
spacenet.add_child(rio)

In [9]:
spacenet.describe()

* <Catalog id=spacenet>
    * <Collection id=rio>


Find all Rio De Janeiro scenes

In [10]:
client = boto3.client('s3')
scenes = client.list_objects(Bucket='spacenet-dataset', Prefix='AOIs/AOI_1_Rio/srcData/mosaic_3band/')['Contents']

For each scene, create and item with a defined bounding box. Each item will include the geotiff as an asset. We will add labels in the next section.

In [11]:
for scene in scenes:
    uri = join('s3://spacenet-dataset/', scene['Key'])
    params = {}
    params['id'] = basename(uri).split('.')[0]
    with rasterio.open(uri) as src:
        params['bbox'] = list(src.bounds)
        params['geometry'] = box(*params['bbox']).__geo_interface__
    params['datetime'] = capture_date
    params['properties'] = {}
    i = Item(**params)
    i.add_asset(key='image', href=uri, title='Geotiff', media_type='image/tiff')
    rio.add_item(i)

In [24]:
spacenet.describe()

* <Catalog id=spacenet>
    * <Collection id=rio>
      * <Item id=013022223103>
      * <Item id=013022223112>
      * <Item id=013022223113>
      * <Item id=013022223121>
      * <Item id=013022223123>
      * <Item id=013022223130>
      * <Item id=013022223131>
      * <Item id=013022223132>
      * <Item id=013022223133>
      * <Item id=013022223301>
      * <Item id=013022223310>
      * <Item id=013022223311>
      * <Item id=013022232002>
      * <Item id=013022232003>
      * <Item id=013022232020>
      * <Item id=013022232021>
      * <Item id=013022232022>
      * <Item id=013022232023>
      * <Item id=013022232200>
      * <Item id=013022232201>


Currently, this STAC only exists in memory. We can specify the uri's using the `set_uris_from_root` method. You give this method a root directory and it will generate paths for all STAC objects relative to that directory.

In [34]:
spacenet.set_uris_from_root('spacenet-stac')

Not, for example, the path of the rio collection json

In [35]:
spacenet.get_children()[0].get_links('self')

[<Link rel=self target=spacenet-stac/rio/collection.json>]

Use the `save` method to save off the JSON file structure

In [23]:
spacenet.save()

### Create new STAC with COGs and labels

Not all of the Spacenet Rio images have labels associates with them. Some fall outside of the labeled AOI. In this case we are only interested in a STAC that includes scenes (i.e. STAC items) that have building labels. We will first check to make sure that each image has labels, if it does, we will create a COG of that image and a new STAC item that points to both the COG and the labels. 

In this case, the labels are all in one large geojson on s3. We will need to read them in and break them into individual label Geojsons for each scene.

In [36]:
labels = gpd.read_file('s3://spacenet-dataset/AOIs/AOI_1_Rio/geojson_buildings/AOI_1_Rio_geojson_buildings.geojson')

In order to create the COGs you will need to install [rio-cogeo](https://github.com/cogeotiff/rio-cogeo).

`$ pip install rio-cogeo`

You can map over each item in a catalog using the `map_items` method. This method takes a user-specified function (`item_mapper`) and maps it over all items within a copy of the catalog. It returns the altered catalog. The `item_mapper` function must take an item and return either another item or a list of items. 

The item mapper defined below checks whether or not the image associated with each item overlaps with the labels. If the image doesn't, it manipulates that item's id to indicate so. If the image does overlap with labels, it saves off the ovelapping labels as a geojson, creates a COG of the image and updates the item to reflect the changes.

In [37]:
def cogify_if_labeled(item, labels, data_dir):
    scene_labels = labels[labels.within(Polygon(item.geometry['coordinates'][0]))]
    if len(scene_labels) == 0:
        item.id += '-no-labels'
    else:
        label_uri = join(data_dir, '{}-labels.geojson'.format(item.id))
        scene_labels.to_file(label_uri, driver='GeoJSON')
        item.add_asset(key='label', href=label_uri, media_type='application/geo+json')
        output_cog_uri = join(data_dir, '{}-cog.tif'.format(item.id))
        call(' '.join(['rio', 'cogeo', 'create', item.assets['image'].href, output_cog_uri]), shell=True)
        item.assets['image'].href = output_cog_uri
    return item

In [38]:
spacenet_cog = spacenet.map_items(cogify_if_labeled, labels=labels, data_dir='~/spacenet-cogs/')

In [39]:
spacenet_cog.describe()

* <Catalog id=spacenet>
    * <Collection id=rio>
      * <Item id=013022223103-no-labels>
      * <Item id=013022223112-no-labels>
      * <Item id=013022223113-no-labels>
      * <Item id=013022223121-no-labels>
      * <Item id=013022223123>
      * <Item id=013022223130>
      * <Item id=013022223131>
      * <Item id=013022223132>
      * <Item id=013022223133>
      * <Item id=013022223301-no-labels>
      * <Item id=013022223310-no-labels>
      * <Item id=013022223311-no-labels>
      * <Item id=013022232002-no-labels>
      * <Item id=013022232003-no-labels>
      * <Item id=013022232020>
      * <Item id=013022232021>
      * <Item id=013022232022>
      * <Item id=013022232023>
      * <Item id=013022232200>
      * <Item id=013022232201-no-labels>


After creating COGs and updating STAC items, we need to filter out the items that do not have labels associated with them. We can create a new list of links and filter the item links by id, then update the links property in the  root catalog.

In [40]:
new_links = []
for link in spacenet_cog.get_child('rio').links:
    if link.rel != 'item':
        new_links.append(link)
    elif not link.target.id.endswith('-no-labels'):
        new_links.append(link)

In [41]:
spacenet_cog.get_child('rio').links = new_links

In [42]:
spacenet_cog.describe()

* <Catalog id=spacenet>
    * <Collection id=rio>
      * <Item id=013022223123>
      * <Item id=013022223130>
      * <Item id=013022223131>
      * <Item id=013022223132>
      * <Item id=013022223133>
      * <Item id=013022232020>
      * <Item id=013022232021>
      * <Item id=013022232022>
      * <Item id=013022232023>
      * <Item id=013022232200>


Finally we set a different root uri for this STAC and save it locally

In [43]:
spacenet_cog.set_uris_from_root('spacenet-cog-stac/')
spacenet_cog.save()